Collaborative

SVD

In [ ]:
#SVD
import numpy as np
import pandas as pd
import time

df = pd.read_csv('rating.csv', header=0)
anime_names = pd.read_csv('anime.csv', header=0)
anime_names = anime_names.drop(['genre','type','episodes','rating','members'], axis=1)
df = pd.merge(df, anime_names, on="anime_id")
df
n_users = df.user_id.nunique()
n_animes = df.anime_id.nunique()

print('Num. of Users: '+ str(n_users))
print('Num of Movies: '+ str(n_animes))


A = df.pivot_table(index='user_id',columns='name',values='rating')

t1 = time.time()

A_T = np.transpose(A)
V = np.dot(A_T, A) 
U = np.dot(A, A_T)
s, v = np.linalg.eig(A) 
s_diag_matrix = np.diag(s)
rating_matrix = X_pred = np.dot(np.dot(U, s_diag_90), v.T)

movie = input('Please enter an anime you like:')
print('Finding anime for you using SVD...')
movie_user_ratings = rating_matrix[movie]
similar_to_movie = rating_matrix.corrwith(movie_user_ratings)

corr_movie = pd.DataFrame(similar_to_movie,columns=['Correlation'])
corr_movie.dropna(inplace=True)
corr_movie = corr_movie.sort_values('Correlation',ascending=False)
corr_movie.drop('Correlation',axis=1,inplace=True)
print('SVD RMSE: ' + str(np.sqrt(rmse(rating_matrix, A))))
print('Anime Recommendations for you:')
corr_movie.head()
print('Time taken:' +t1)

Num. of Users: 51999
Num of Movies: 11107


In [ ]:
#SVD with 90% energy retention
t2 = time.time()

print('Finding anime for you using SVD with 90% energy retention...')

A_T = np.transpose(A)
V = np.dot(A_T, A) 
U = np.dot(A, A_T)
s, v = np.linalg.eig(m) 
s_diag_matrix = np.diag(s)

m = 0.9 * len(s)
s_90 = s[0:m]
s_diag_90 = np.diag(s_90)
rating_matrix_90 = np.dot(np.dot(u, s_diag_90), vt)

movie_user_ratings_90 = rating_matrix_90[movie]
similar_to_movie_90 = rating_matrix_90.corrwith(movie_user_ratings)

corr_movie = pd.DataFrame(similar_to_movie_90,columns=['Correlation'])
corr_movie.dropna(inplace=True)
corr_movie = corr_movie.sort_values('Correlation',ascending=False)
corr_movie.drop('Correlation',axis=1,inplace=True)
print('SVD_90 RMSE: ' + str(np.sqrt(rmse(rating_matrix_90, A))))
print('Anime Recommendations for you:')
corr_movie.head()
print('Time taken:' +t2)

CUR

In [ ]:
import numpy as np
import pandas as pd
import time

In [ ]:
def diag_inv(mat):
    #Pseudo Diagonal matrix needed for 'U' matrix
    for x in range(mat.shape[0]):
        if mat[x,x] != 0:
            mat[x,x] = 1/mat[x,x]
    
    return mat

def error(A, C, U, R):
    #Frobenius norm (error) between the decomposed matrix and original matrix
    A_pseudo = np.matmul(C, np.matmul(U, R))
    frob_norm = np.linalg.norm(A - A_pseudo)
    return frob_norm

In [ ]:
def select(arr):

    #User defined input to choose the number of rows and columns he/she wants to select    
    c = int(input('Enter the number of columns to be selected: '))
    r = int(input('Enter the number of rows to be selected: '))
    mod = input('Enable modified CUR (removes duplicate rows and columns) [Y/N]: ')
    
    #Sum of squares of all entries
    sqr_sum = np.linalg.norm(arr)**2
    
    #t0 calculate the time taken to perform CUR decomposition for each step C U and R
    t1 = time.time()

    #C-matrix
    score_col = np.sum(np.square(arr), axis = 0)/sqr_sum
    if mod.lower() == 'y':
        score_col = score_col.reshape(1, len(score_col))
        _, ind = np.unique(score_col, axis = 1, return_index = True)
        for x in range(score_col.shape[1]):
            if x in ind:
                continue
            else:
                score_col[:,x] = 0
        score_col = score_col.reshape(score_col.shape[1], )
    indices_col = score_col.argsort()[-c:][::-1]
    c_mat = arr[:,indices_col]/np.sqrt(c*score_col[indices_col])
    
    t2 = time.time()
    
    #R_matrix
    score_row = np.sum(np.square(arr), axis = 1)/sqr_sum
    if mod.lower() == 'y':
        score_row = score_row.reshape(1, len(score_row))
        _, ind = np.unique(score_row, axis = 1, return_index = True)
        for x in range(score_row.shape[1]):
            if x in ind:
                continue
            else:
                score_row[:,x] = 0
        score_row = score_row.reshape(score_row.shape[1], )
    indices_row = score_row.argsort()[-r:][::-1]
    r_mat = arr[indices_row,:]/np.sqrt(r*score_row[indices_row].reshape(len(score_row[indices_row]),1))
    
    t3 = time.time()

    #U - matrix
    k = arr[indices_row,:]
    A = k[:,indices_col]
    A_T = np.transpose(A)
    V = np.dot(A_T, A) 
    U = np.dot(A, A_T)
    s, v = np.linalg.eig(A) 
    s_diag_matrix = np.diag(s)
    #rating_matrix = X_pred = np.dot(np.dot(u, s_diag_90), vt)
    
    u_mat = np.matmul(v, np.matmul(np.square(diag_inv(s_diag_matrix)), u.T))
    
    t4 = time.time()
    
    print("Time taken for C-matrix construction selection is... " + str(t2-t1) + "/n")
    print("Time taken for R-matrix construction selection is... " + str(t3-t2) + "/n")
    print("Time taken for U-matrix construction selection is... " + str(t4-t3) + "/n")
    
    #Frobenius norm for decomposed matrix
    frob_norm = error(arr, c_mat, u_mat, r_mat)
    
    print("The frobenius norm of the CUR decomposition is: " + str(frob_norm))
    
    return c_mat, u_mat, r_mat

In [ ]:
#Given Input and target variable the correlation matrix using pearson formula is calculated
def pearson_coeff(X, Y):
    data = np.hstack((X,Y))
    dframe = pd.DataFrame(data)
    corr_matrix = dframe.corr(method = 'pearson')
    return corr_matrix

In [ ]:
def rmse(act , pred):
    mat = a - a_cap
    return np.sqrt(np.sum(np.square(mat))/len(a))

In [ ]:
def k_precision(act, pred, k_val):
    act_val = []
    pred_val = []
    n_rel = 0
    for i in range(len(act)):
            if act[i] >= k_val:
                act_val.append(act[i])
                pred_val.append(pred[i])
        act_val = np.array(act_val)
        pred_val = np.array(pred_val)
    ind = act_val.argsort()[::-1]
    act_val =  act_val[ind]
    pred_val = pred_val[ind]

    for x in range(len(act_val)):
        if pred_val[x] < k_val:
            n_rel = n_rel + 1

    precision = (len(act_val) - n_rel)/len(act_val)

    return precision
